In [143]:
import pandas as pd
import numpy as np
import re

In [144]:
df = pd.read_csv('data_final.csv')

In [145]:
df.shape

(37790, 6)

In [146]:
df.head()

,ID commande,Nom du fabricant,Product Category Default,Article,Code postal (Facture),Ville (Facture)
0,1743,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250.0,SAINT PALAIS
1,1743,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250.0,SAINT PALAIS
2,1743,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250.0,SAINT PALAIS
3,1743,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250.0,SAINT PALAIS
4,1743,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250.0,SAINT PALAIS


## 1 Modification du nom des colonnes

In [147]:
df.columns

Index(['ID commande', 'Nom du fabricant', 'Product Category Default',
       'Article', 'Code postal (Facture)', 'Ville (Facture)'],
      dtype='object')

In [148]:
df.columns = ['commande','producteur','categorie','nom_produit','code_postal','commune']

## 2 Modification contenu code_postal

In [149]:
df.code_postal.dtypes

dtype('float64')

In [150]:
df.code_postal = df.code_postal.astype(int)

In [151]:
df.dtypes

commande        int64
producteur     object
categorie      object
nom_produit    object
code_postal     int64
commune        object
dtype: object

In [152]:
df.head()

,commande,producteur,categorie,nom_produit,code_postal,commune
0,1743,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250,SAINT PALAIS
1,1743,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250,SAINT PALAIS
2,1743,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250,SAINT PALAIS
3,1743,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250,SAINT PALAIS
4,1743,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250,SAINT PALAIS


In [153]:
df.code_postal.unique()

array([64250, 64240, 64780, 64480, 64640, 64100, 64990, 64600, 64310,
       64210, 64200, 64120, 40220, 64122, 40440, 77700, 64340, 35580,
       40530, 31300, 24270, 64500, 38000, 64330, 64190, 64140, 64220,
       79400, 40300, 64300, 78000, 64259, 40390])

In [154]:
len(df.commande.unique())

5270

In [155]:
df.shape

(37790, 6)

In [156]:

len(df.nom_produit.unique())

311

In [157]:
df[df.categorie == 'FROMAGES']['nom_produit'].unique()

array(["fromage de brebis fermier (650g environ) +6 mois d'affinage.",
       'Fromage Pérail de brebis.',
       "fromage de brebis fermier (1.5kg environ) +6 mois d'affinage.",
       'Sarrin frais.', 'Espil.', 'Garralda.', 'Ixuri.', 'Xedarri.',
       'Patin.',
       "fromage de brebis fermier (1.5 kg environ) -6 mois d'affinage.",
       "fromage de brebis fermier (650g environ) -6 mois d'affinage.",
       "fromage de brebis fermier (500g environ) 10/12 mois d'affinage.",
       "fromage de brebis fermier (1kg environ) 10/12 mois d'affinage.",
       "fromage de brebis fermier (3KG) - 6 mois d'affinage.",
       'Etxebazter.', 'Manex.'], dtype=object)

In [158]:
len(df[df.categorie == 'FROMAGES']['nom_produit'].unique())

16

In [159]:
for cat in df.categorie.unique():
    for produit in df[df.categorie == cat]['nom_produit'].unique():
        print(cat,produit)

CRÈMERIE breuil (375g).
CRÈMERIE Yaourt nature bio au lait entier (4 x125 gr).
CRÈMERIE Fromage blanc de vache nature,lisse bio (500g).
CRÈMERIE Yaourt a la fraise bio au lait entier  (4 x 125 gr).
CRÈMERIE Beurre doux bio au lait cru ( 250 gr).
CRÈMERIE Creme fraiche bio  au lait cru (250ml).
CRÈMERIE Lait de vache demi-ecreme pasteurise bio (1L).
CRÈMERIE Yaourt a la framboise bio  au lait entier (4 x125 gr).
CRÈMERIE Fromage blanc de vache nature campagne bio (500g).
CRÈMERIE Yaourt a la myrtille bio  au lait entier (4 x 125g).
CRÈMERIE Breuil (1kg).
CRÈMERIE Lait de vache entier pasteurise bio (1L).
CRÈMERIE Mamia (x4).
CRÈMERIE Breuil (500g).
CRÈMERIE Yaourt au citron, bio,  au lait entier (4 x 125g).
CRÈMERIE Beurre salé bio au lait cru (250g).
CRÈMERIE Yaourt a la vanille  bio  au lait entier ( 4 x 125 gr).
CRÈMERIE Yaourt brebis nature (lot de 4).
CRÈMERIE Yaourt brebis fruits rouges coulis (lot de 4).
CRÈMERIE Yaourt aux fruits, bio, au lait entier (4 x 125g).
CRÈMERIE Yaourt 

## 3 Reduction du nombre de produits
Actuellement il y a 311 produits différents, mais on remarque que l'on peut réunir en un seul plusieurs produits (par exemple il y a 7 "fromages de brebis fermier" car avec des poids différents...)

### Pour la catégorie CRÈMERIE

In [160]:
print(df[df.categorie == 'CRÈMERIE']['nom_produit'].unique(), len(df[df.categorie == 'CRÈMERIE']['nom_produit'].unique()))

['breuil (375g).' 'Yaourt nature bio au lait entier (4 x125 gr).'
 'Fromage blanc de vache nature,lisse bio (500g).'
 'Yaourt a la fraise bio au lait entier  (4 x 125 gr).'
 'Beurre doux bio au lait cru ( 250 gr).'
 'Creme fraiche bio  au lait cru (250ml).'
 'Lait de vache demi-ecreme pasteurise bio (1L).'
 'Yaourt a la framboise bio  au lait entier (4 x125 gr).'
 'Fromage blanc de vache nature campagne bio (500g).'
 'Yaourt a la myrtille bio  au lait entier (4 x 125g).' 'Breuil (1kg).'
 'Lait de vache entier pasteurise bio (1L).' 'Mamia (x4).'
 'Breuil (500g).' 'Yaourt au citron, bio,  au lait entier (4 x 125g).'
 'Beurre salé bio au lait cru (250g).'
 'Yaourt a la vanille  bio  au lait entier ( 4 x 125 gr).'
 'Yaourt brebis nature (lot de 4).'
 'Yaourt brebis fruits rouges coulis (lot de 4).'
 'Yaourt aux fruits, bio, au lait entier (4 x 125g).'
 'Yaourt brebis citron (lot de 4).' 'Lait de chevre AB (1L).'
 'Fromage blanc de chevre (500g).' 'Fromage blanc de chevre (420g).'] 24


On peut réduire le nombre actuels de produits de 24 à 7
- breuil
- mamia
- yaourt
- fromage blanc
- lait
- beurre
- creme fraiche

In [161]:
filtre1 = (df.categorie == 'CRÈMERIE') | (df.categorie == 'YAOURT & FROMAGE BLANC')
df_cremerie_yaourt = df[filtre1].copy()

In [162]:
df_cremerie_yaourt

,commande,producteur,categorie,nom_produit,code_postal,commune
0,1743,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250,SAINT PALAIS
3,1743,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250,SAINT PALAIS
6,1750,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64240,HASPARREN
7,1750,FERME ESNEKIAK,CRÈMERIE,"Fromage blanc de vache nature,lisse bio (500g).",64240,HASPARREN
8,1750,FERME ESNEKIAK,CRÈMERIE,Yaourt a la fraise bio au lait entier (4 x 12...,64240,HASPARREN
...,...,...,...,...,...,...
37780,7205,FERME ESNEKIAK,CRÈMERIE,"Yaourt au citron, bio, au lait entier (4 x 12...",64480,ustaritz
37781,7205,FERME ESNEKIAK,CRÈMERIE,Yaourt a la fraise bio au lait entier (4 x 12...,64480,ustaritz
37782,7205,FERME ESNEKIAK,CRÈMERIE,Yaourt a la myrtille bio au lait entier (4 x ...,64480,ustaritz
37787,7205,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64480,ustaritz


In [163]:
def product_cremerie_yaourt(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'yaourt' in nom:
        return 'yaourt'
    elif 'breuil' in nom:
        return 'breuil'
    elif 'mamia' in nom:
        return 'mamia'
    elif 'fromage blanc' in nom:
        return 'fromage blanc'
    elif 'beurre' in nom:
        return 'beurre'
    elif 'creme fraiche' in nom:
        return 'creme fraiche'
    elif 'lait' in nom:
        return 'lait'
    else:
        return 'erreur'

df_cremerie_yaourt['produit'] = df_cremerie_yaourt.apply(product_cremerie_yaourt, axis=1)

In [164]:
df_cremerie_yaourt['produit'].unique()

array(['breuil', 'yaourt', 'fromage blanc', 'beurre', 'creme fraiche',
       'lait', 'mamia'], dtype=object)

### Catégorie Fromages et Fromage de vache
Pour simplifier nous réunissons le traitement de ces deux catégories

In [165]:
df.categorie.unique()

array(['CRÈMERIE', 'FROMAGES', 'FRUITS & LÉGUMES', 'ÉPICERIE', 'FRUITS',
       'VOLAILLES & OEUFS', 'VIANDES', 'VOLAILLES', 'ÉPICERIE SUCRÉE',
       'YAOURT & FROMAGE BLANC', 'VIANDE FRAÎCHE', 'ÉPICERIE SALÉE',
       'LÉGUMES', 'BOISSONS', 'NOS PRODUITS', 'FROMAGE DE VACHE',
       'PLATS CUISINÉS'], dtype=object)

In [166]:
filtre = (df.categorie == 'FROMAGES') | (df.categorie == 'FROMAGE DE VACHE')

In [167]:
df_fromage = df[filtre].copy()

In [168]:
def product_fromage(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'fromage de brebis fermier' in nom:
        return 'fromage de brebis fermier'
    elif 'fromage pérail de brebis' in nom:
        return 'fromage pérail de brebis'
    elif 'tomme de vache fraîche' in nom:
        return 'tomme de vache fraîche'
    elif 'fromage de vache affiné' in nom:
        return 'fromage de vache affiné'
    elif 'sarrin' in nom or 'espil' in nom or 'garralda' in nom or 'ixuri' in nom or'xedarri' in nom or 'manex' in nom or 'patin' in nom or 'etxebazter' in nom:
        return 'fromage de chèvre'
    else:
        return 'erreur'


df_fromage['produit'] = df_fromage.apply(product_fromage, axis = 1)

In [169]:
df_fromage[df_fromage.produit == 'erreur']

,commande,producteur,categorie,nom_produit,code_postal,commune,produit


### Catégorie Fruits & légumes et catégorie fruits
Pour simplifier nous réunissons le traitement de ces deux catégories également

In [170]:
filtre2 = (df.categorie == 'FRUITS & LÉGUMES') | (df.categorie == 'FRUITS')| (df.categorie == 'NOS PRODUITS')

In [171]:
df_fruits = df[filtre2].copy()

In [172]:
df_fruits['nom_produit'].unique()

array(['radis noir (la botte)', 'betterave rouge (la botte)',
       'Pommes Gordin Xuri (1kg)', 'courge potimarron ou butternut (1kg)',
       'courge longue de naples (1kg)',
       "Pomme de terre de l'année(2.5kg)", 'Persil (la botte)',
       'batavia ou feuille de chene',
       'assortiment chou fleur / romanesco (0,8-1kg)', 'Mache AB (200gr)',
       'chou frise ou cabus (600gr a 800gr)', 'Epinard (300g)',
       'chicoree scarole', 'choux de Bruxelles (500g)',
       'pomme de terre vapeur (2kg)', 'navet (750g)',
       "Pomme de terre de l'année (1kg)", 'chou de chine',
       'Cagette "surprise"', "Poireaux d'HIVER (lot de 3 ou 4)",
       'Fenouil (500gr)',
       'Céleri branche (le bouquet de 5 ou 6 branches)',
       'Oignon  rouge ( la botte)', 'Ail sec (300gr)',
       'Blette (la botte)', 'feves (1-1,2kg)', 'petit pois Mangetout',
       'pomme de terre primeur 1kg', 'Cerises Bigarreau (600 gr)',
       'Cerises Bigarreau (300 gr)', 'radis de printemps (la botte)',
  

In [173]:
len(df_fruits['nom_produit'].unique())

91

In [174]:
def product_fruits_legumes(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'radis' in nom:
        return 'radis'
    elif 'betterave' in nom:
        return 'betterave'
    elif 'melon' in nom:
        return 'melon'
    elif 'pomme de terre' in nom or 'patate' in nom:
        return 'pomme de terre'
    elif 'pommes' in nom or 'sagarra' in nom:
        return 'pommes'
    elif 'persil' in nom:
        return 'persil'
    elif ('batavia' in nom)|('mache' in nom)|('scarole' in nom)|('roquette' in nom) or 'mesclun' in nom:
        return 'salade'
    elif 'chou' in nom:
        return 'chou'
    elif 'oignon' in nom:
        return 'oignon'
    elif 'courge' in nom or 'musquée de provence' in nom or 'longue de nice' in nom or 'butternut' in nom:
        return 'courge'
    elif 'epinard' in nom:
        return 'epinard'
    elif 'navet' in nom:
        return 'navet'
    elif 'cagette' in nom:
        return 'assortiment'
    elif 'poireaux' in nom:
        return 'poireaux'
    elif 'fenouil' in nom:
        return 'fenouil'
    elif 'blette' in nom:
        return 'blette'
    elif 'ail sec' in nom or 'aillet' in nom:
        return 'ail'
    elif 'céleri' in nom or 'celeri' in nom:
        return 'celeri'
    elif 'feves' in nom:
        return 'feves'
    elif 'petit pois' in nom:
        return 'petit pois'
    elif 'cerises' in nom:
        return 'cerises'
    elif 'agrumes' in nom or 'clementines' in nom or 'oranges' in nom or 'citron' in nom or 'pomelos' in nom:
        return 'agrumes'
    elif 'piment' in nom:
        return 'piment'
    elif 'concombre' in nom:
        return 'concombre'
    elif 'carotte' in nom:
        return 'carotte'
    elif 'tomate' in nom:
        return 'tomate'
    elif 'aubergine' in nom:
        return 'aubergine'
    elif 'kiwi' in nom:
        return 'kiwi'
    elif 'echalote' in nom:
        return 'echalote'
    elif 'poivron' in nom:
        return 'poivron'
    elif 'coffret' in nom:
        return 'autre'
    elif 'pastèque' in nom:
        return 'pastèque'
    elif 'ciboulette' in nom:
        return 'ciboulette'
    elif 'potimarron' in nom:
        return 'potimarron'
    elif 'haricots verts' in nom:
        return 'haricots verts'
    elif 'pêches' in nom:
        return 'pêches'
    elif 'patidou' in nom:
        return 'patidou'
    else:
        return 'erreur'



df_fruits['produit'] = df_fruits.apply(product_fruits_legumes,axis=1)

In [175]:
len(df_fruits['produit'].unique())

36

In [176]:
df_fruits[df_fruits.produit == 'erreur']

,commande,producteur,categorie,nom_produit,code_postal,commune,produit


### Catégorie ÉPICERIE + EPICERIE SUCREE + EPICERIE SALEE

In [177]:
filtre3 = (df.categorie == 'ÉPICERIE')|(df.categorie == 'ÉPICERIE SUCRÉE')|(df.categorie == 'ÉPICERIE SALÉE')
df_epicerie = df[filtre3].copy()

In [178]:
df_epicerie['nom_produit'].unique(), len(df_epicerie['nom_produit'].unique())

(array(["BIZIGAI Piment d'Espelette AOP (50 gr)",
        'Confiture de pêches (360g)', 'compote de pommes 750g',
        'Coulis de Mûres (320g)', 'Jus de pommes (1L)',
        'Confiture de Figue (360g)', 'Coulis de Cassis (320g)',
        'Soupe de poisson (1L)',
        'Miel de bruyère cendrée de la forêt landaise (pot de 500g)',
        "Sirop de Fleurs d'Accacia (320ml)",
        "Confiture de cerise d'Itxassou (260 gr)",
        'Farine de Maïs Grand Roux Basque AB (500 gr)',
        "Purée de Piment d'Espelette (95 gr) SUGORRI",
        'Miel de Châtaigner (pot de 500g)', 'gelée de pommes Anisa 190g',
        'Miel fleurs de printemps (500g)',
        "Gelée de Pommes au Piment d'Espelette (125 gr) SAGARMIN",
        'Confiture de lait de chèvre (100 gr)',
        'Confiture de prune (250 gr)', 'Farine bise semi complète (1kg)',
        'Miel de Callune de la montagne du Pays Basque (pot de 250g)',
        'CIDRE FERMIER (1l)', 'Sirop menthe-citron (320ml)',
        "Pain d'ép

In [179]:
def product_epicerie(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'confiture' in nom or 'gelée' in nom:
        return 'confiture'
    elif 'savon' in nom:
        return 'savon'
    elif 'vin' in nom or 'cidre' in nom:
        return 'vin ou cidre'
    elif 'compote' in nom:
        return 'compote'
    elif 'coulis' in nom:
        return 'coulis'
    elif 'nectar' in nom or 'jus' in nom or 'sirop' in nom:
        return 'jus ou sirop'
    elif 'farine' in nom:
        return 'farine'
    elif 'bizigai' in nom or 'purée' in nom:
        return 'piment'
    elif 'soupe' in nom:
        return 'soupe'
    elif 'crème' in nom or 'pâte à tartiner' in nom:
        return 'pâte à tartiner'
    elif "pain d'épices" in nom:
        return "pain d'épices"
    elif 'miel' in nom:
        return 'miel'
    elif 'pollen' in nom:
        return 'pollen'
    elif 'polenta' in nom:
        return 'polenta'
    elif 'huile' in nom:
        return 'huile'
    elif 'fusilli' in nom or 'coquillettes' in nom:
        return 'pâtes'
    else:
        return 'erreur'
df_epicerie['produit'] = df_epicerie.apply(product_epicerie,axis=1)

In [180]:
df_epicerie.shape[0], df_epicerie[df_epicerie.produit == 'erreur'].shape[0]

(4241, 0)

In [181]:
len(df_epicerie.produit.unique())

16

In [182]:
df.categorie.unique()

array(['CRÈMERIE', 'FROMAGES', 'FRUITS & LÉGUMES', 'ÉPICERIE', 'FRUITS',
       'VOLAILLES & OEUFS', 'VIANDES', 'VOLAILLES', 'ÉPICERIE SUCRÉE',
       'YAOURT & FROMAGE BLANC', 'VIANDE FRAÎCHE', 'ÉPICERIE SALÉE',
       'LÉGUMES', 'BOISSONS', 'NOS PRODUITS', 'FROMAGE DE VACHE',
       'PLATS CUISINÉS'], dtype=object)

## Catégorie Volailles & oeufs + Volailles + plats cuisinés

In [183]:
filtre4 = (df.categorie == 'VOLAILLES & OEUFS') | (df.categorie == 'VOLAILLES')| (df.categorie == 'PLATS CUISINÉS')
df_volailles = df[filtre4].copy()


In [184]:
df_volailles['nom_produit'].unique(), len(df_volailles['nom_produit'].unique())

(array(['Petit poulet fermier 1,250 kg (+/- 100 gr) AB.',
        'Petite pintade fermière 1,250 kg (+/- 100gr) AB.',
        'Poulet fermier moyen 1,6 kg (+/-100 gr) AB.',
        'Pintade fermière moyenne 1,5 kg (+/- 100gr) AB.',
        'Grosse pintade fermière 1,8 kg (+/- 100gr) AB.',
        'Gros poulet fermier 1,8 kg (+/- 100gr) AB.',
        'Poule fermière 1,300 kg (+/- 100 gr) AB.',
        'Oeufs fermiers AB (x6)', 'Oeufs fermiers AB (x12)',
        'Oeufs fermiers AB (x10)', 'Oeufs fermiers AB (x12) T',
        'Poulet fermier moyen 1,6 kg (+/-100 gr) AB.M',
        'Oeufs fermiers AB (x6) L', 'Oeufs fermiers AB (x12) L',
        'Poulet fermier moyen (1,6 - 1,8 kg).X',
        'Gros poulet fermier (1,8 - 2,0 kg).X', 'Poule confite'],
       dtype=object),
 17)

In [185]:
def product_volailles(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'poulet' in nom:
        return 'poulet'
    elif 'pintade' in nom:
        return 'pintade'
    elif 'poule' in nom:
        return 'poule'
    elif 'oeufs' in nom:
        return 'oeufs'
    else:
        return 'erreur'
df_volailles['produit'] = df_volailles.apply(product_volailles,axis=1)

In [186]:
df_volailles.shape[0], df_volailles[df_volailles.produit == 'erreur'].shape[0]

(1453, 0)

## Catégories viandes et viande fraiche

In [187]:
filtre5 = (df.categorie == 'VIANDES') | (df.categorie == 'VIANDE FRAÎCHE')
df_viande = df[filtre5].copy()

In [188]:
df_viande['nom_produit'].unique(), len(df_viande['nom_produit'].unique())

(array(['Axoa de veau bio  (750g)', 'Colis de porc Kintoa (4,500kg).',
        'Saucisses de Porc Basque x4 (400 gr).',
        'Cotes (échine) de Porc Basque x2 (400 gr).',
        'Cotes (filet) de Porc Basque x2 (350 gr).',
        'Foie de veau AB (300 gr).', 'Tendrons de veau AB (1 kg).',
        'Sauté de veau AB (1 kg).', 'Cotes de veau AB (500 gr).',
        'Tendrons de veau AB (500 gr).', 'Sauté de veau AB (500 gr).',
        'Escalopes de veau AB (500 gr).', 'Cotes de veau AB (1 kg).',
        'agneau de lait (entre 6-8kg) BIXARTEA.',
        'demi agneau de lait (4-4.5kg) BIXARTEA.',
        'Ventrêche de Porc Basque (250 gr).', 'Pate de porc basque (190g)',
        'Chichon de porc basque (180g)',
        'Colis de 3Kg de steacks hachés bio ( environ 25 steacks).',
        'Boudin de porc basque (190 gr)',
        '4 steacks hachés bio ( environ  500gr).',
        '8 steacks hachés bio ( environ  1 Kg).',
        'Canette Kriaxera à rotir (2,9 kg - 3,1 kg).',
        'Demi

In [189]:
def product_viande(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'veau' in nom:
        return 'viande de veau'
    elif 'porc' in nom:
        return 'viande de porc'
    elif 'chèvre' in nom or 'chevreau':
        return 'viande de chèvre ou chevreau'
    elif 'canard' in nom:
        return 'viande de canard'
    elif 'boeuf' in nom or 'steacks hachés' in nom:
        return 'viande de boeuf'
    elif 'agneau' in nom:
        return "viande d'agneau"
    elif 'thon' in nom or 'poisson' in nom:
        return 'poisson'
    elif 'pigeonneau' in nom:
        return 'pigeonneau'
    else:
        return 'erreur'
df_viande['produit'] = df_viande.apply(product_viande,axis=1)

In [190]:
df_viande.shape[0], df_viande[df_viande.produit == 'erreur'].shape[0]

(2699, 0)

## Catégorie boissons

In [191]:
filtre6 = df.categorie == 'BOISSONS'
df_boissons = df[filtre6].copy()

In [192]:
df_boissons.nom_produit.unique()

array(['Tisane Detox (25 gr)', 'Tisane Air Pur (25 gr)',
       'Tisane Souplesse (25 gr)', 'Tisane Aube (25 gr)',
       'Tisane la Zen (25 gr)', 'Tisane coquelicots (25 gr)',
       'Tisane nuit étoilée (25 gr)', 'Tisane la gloutonne (25g)'],
      dtype=object)

In [193]:
def product_boissons(row):
    return 'tisane'
df_boissons['produit'] = df_boissons.apply(product_boissons,axis=1)

In [194]:
df_boissons.shape[0], df_boissons[df_boissons.produit == 'erreur'].shape[0]

(138, 0)

array(['Patate douce 1kg', 'Coffret de Noël'], dtype=object)

## On peut remplir la colonne "produit" de notre df
On appelle chaque fonction selon la categorie du produit

In [195]:
def fill_product(row):
    if ((row.categorie == 'CRÈMERIE') | (row.categorie == 'YAOURT & FROMAGE BLANC')):
        return product_cremerie_yaourt(row)
    elif ((row.categorie == 'FROMAGES') | (row.categorie == 'FROMAGE DE VACHE')):
        return product_fromage(row)
    elif ((row.categorie == 'FRUITS & LÉGUMES') | (row.categorie == 'FRUITS')| (row.categorie == 'NOS PRODUITS')| (row.categorie == 'LÉGUMES')):
        return product_fruits_legumes(row)
    elif ((row.categorie == 'ÉPICERIE') | (row.categorie == 'ÉPICERIE SUCRÉE')| (row.categorie == 'ÉPICERIE SALÉE')):
        return product_epicerie(row)
    elif ((row.categorie == 'VOLAILLES & OEUFS') | (row.categorie == 'VOLAILLES')| (row.categorie == 'PLATS CUISINÉS')):
        return product_volailles(row)
    elif ((row.categorie == 'VIANDES') | (row.categorie == 'VIANDE FRAÎCHE')):
        return product_viande(row)
    elif (row.categorie == 'BOISSONS'):
        return 'tisane'
df['produit'] = df.apply(fill_product, axis=1)

In [196]:
# On vérifie s'il y a des NA ou 'erreur' dans la nouvelle colonne
len(df[df.produit == 'erreur']), len(df[pd.isna(df.produit)])

(0, 0)

In [197]:
len(df.produit.unique())

72

Nous avons réduit le nombre de produits à 72, il était de 311 initialement.

In [198]:
df.head()

,commande,producteur,categorie,nom_produit,code_postal,commune,produit
0,1743,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250,SAINT PALAIS,breuil
1,1743,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250,SAINT PALAIS,fromage de brebis fermier
2,1743,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250,SAINT PALAIS,radis
3,1743,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250,SAINT PALAIS,yaourt
4,1743,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250,SAINT PALAIS,piment


In [199]:
df_to_transform = df[['commande','produit']]

In [200]:
df_to_transform

,commande,produit
0,1743,breuil
1,1743,fromage de brebis fermier
2,1743,radis
3,1743,yaourt
4,1743,piment
...,...,...
37785,7205,vin ou cidre
37786,7205,jus ou sirop
37787,7205,yaourt
37788,7205,lait


In [201]:
df_final = pd.pivot_table(df_to_transform, index = 'commande', values ='produit', columns='produit', aggfunc='size',fill_value=0)

In [202]:
df_final

produit,agrumes,ail,assortiment,aubergine,autre,betterave,beurre,blette,breuil,carotte,...,savon,soupe,tisane,tomate,tomme de vache fraîche,viande de chèvre ou chevreau,viande de porc,viande de veau,vin ou cidre,yaourt
commande,,,,,,,,,,,,,,,,,,,,,
1743,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1750,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1752,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
7202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
